<a href="https://colab.research.google.com/github/rjlobosco/elem_fini_2022/blob/main/Mecflu_Dataframes_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTAÇÕES E INSTALAÇÕES

!pip install openpyxl
import math
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# CONSTANTES, INPUTS E FUNÇÕES


# Constantes
PA = 101.325  # Pressão atmosférica ao nível do mar em kPa
k = 1.15e-4   # Constante em s²/m²
g = 9.81      # Aceleração gravitacional em m/s²

# Função para calcular a altura a partir da pressão
def calcular_altura(PA, k, P_h, g):
    # P_h: Pressão a uma altura h (em KPa)
    altura = -math.log(P_h / PA) / (k * g)
    return altura

# Função para converter horário de Brasília para UTC
def brasilia_to_utc(hour_brasilia):
    brasilia_time = datetime.strptime(hour_brasilia, '%H:%M')
    utc_time = brasilia_time + timedelta(hours=3)
    return utc_time.strftime('%H:%M')


# Função para converter temperatura de Kelvin para Celsius
def kelvin_to_celsius(kelvin):
    return kelvin - 273.15


# Níveis de pressão em KPa
pressure_levels = [50.0, 55.0, 60.0, 70.0, 75.0, 77.5, 80.0, 82.5, 85.0, 87.5, 90.0, 92.5, 95.0, 97.5, 100.0]

# Níveis de pressão em hPa
#pressure_levels_hpa = [p * 10 for p in pressure_levels]

# Horas
horas = ['03:00', '09:00', '15:00', '21:00']
#horas_utc = [brasilia_to_utc(h) for h in horas]
#print(horas_utc)


# Coordenadas das plataformas
platforms = {
    'NAMORADO 2 (PNA-2)': (-22.45073, -40.41175),
    'PETROBRAS 26 (P-26)': (-22.4684, -40.02869),
    'PETROBRAS 32 (P-32)': (-22.2051, -40.1431),
    'PETROBRAS 37 (P-37)': (-22.4868, -40.09779),
    'PETROBRAS IX': (-22.57358, -40.82192),
    'PETROBRAS XIX': (-22.3927, -40.05438),
    'PETROBRAS XXXIII': (-22.37, -40.0267),
    'VERMELHO 1 (PVM-1)': (-22.16065, -40.27872),
    'VERMELHO 2 (PVM-2)': (-22.17535, -40.29147),
}

season_names = {
        'DJF': 'Verão',
        'MAM': 'Outono',
        'JJA': 'Inverno',
        'SON': 'Primavera'
        }


In [ ]:
# Cria um DataFrame vazio para armazenar os resultados combinados de todas as pressões
combined_df = pd.DataFrame()

# Loop sobre cada nível de pressão
for level in pressure_levels:

  # Cria um DataFrame vazio para armazenar os resultados combinados de todas as estações do ano
  horas_df = pd.DataFrame()

  for hora in horas:

    hora_utc = brasilia_to_utc(hora)

    # Nome do arquivo NetCDF para o nível de pressão atual com o horário de Brasília
    nc_filename = 'Era5_2023_CAMPOS.nc'

    # Carrega o arquivo NetCDF usando xarray
    ds = xr.open_dataset(nc_filename, engine='h5netcdf')

    # Renomeia a coordenada 'valid_time' para 'time'
    ds = ds.rename({'valid_time': 'time'})

    # Filtra para o horário e pressão desejadas
    filtra_hora = ds.time.dt.strftime('%H:%M') == hora_utc
    filtra_pressao = ds['pressure_level'] == level*10
    # Aplicar ambos os filtros com where
    ds = ds.where(filtra_hora & filtra_pressao, drop=True)

    u = ds['u']
    v = ds['v']

    # Calcula a velocidade do vento (wspd)
    wspd = (ds['u']**2 + ds['v']**2)**0.5

    # Calcula a média da velocidade do vento por estação do ano
    u_mean_season = u.groupby('time.season').mean(dim=['time'])
    v_mean_season = v.groupby('time.season').mean(dim=['time'])
    # Calcula a média da velocidade do vento por estação do ano
    wspd_mean_season = wspd.groupby('time.season').mean(dim=['time'])

    # Calcula a média da temperatura por estação do ano
    temp_mean_season = ds['t'].groupby('time.season').mean(dim=['time'])

    # Cria um DataFrame vazio para armazenar os resultados combinados de todas as estações do ano
    estacoes_df = pd.DataFrame()

    # Loop sobre as estações do ano
    for season in ['DJF', 'MAM', 'JJA', 'SON']:

      # Cria um DataFrame para armazenar os resultados específicos para a estação do ano atual
      estacao_df = pd.DataFrame()

      # Define o nome do índice para as plataformas
      estacao_df.index.name = 'Índice'

      for platform, (lat, lon) in platforms.items():

        # Interpola o valor das componentes da velocidade do vento
        wind_value_u = u_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()
        wind_value_v = v_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()
        # Interpola o valor da média da velocidade do vento
        wind_value = wspd_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

        # Interpola o valor da média da temperatura em Kelvin
        temp_kelvin = temp_mean_season.sel(season=season).interp(latitude=lat, longitude=lon, method='nearest').values.item()

        # Converte a temperatura para Celsius
        temp_celsius = kelvin_to_celsius(temp_kelvin)

        # Adiciona uma linha para cada plataforma
        estacao_df = pd.concat([estacao_df, pd.DataFrame({
                  'Plataforma': platform,
                  'Velocidade_Vento_u_m/s': wind_value_u,
                  'Velocidade_Vento_v_m/s': wind_value_v,
                  'Velocidade_Vento_media_m/s': wind_value,
                  'Temperatura_K': temp_kelvin,
                  'Temperatura_C': temp_celsius,
              }, index=[0])], ignore_index=True)


      # Armazena o nome da estação na coluna 'Estação_do_Ano'
      estacao_df['Estação_do_Ano'] = season_names[season]

      # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
      estacoes_df = pd.concat([estacoes_df, estacao_df], ignore_index=True)



    #Adiciona as colunas de hora
    estacoes_df['Horário_Brasília'] = hora
    estacoes_df['Horário_UTC'] = hora_utc

    # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
    horas_df = pd.concat([horas_df, estacoes_df], ignore_index=True)

  # Adiciona a coluna de pressão
  horas_df['Nível_de_Pressão_hPa'] = int(level*10)

  # Calcula a altura correspondente ao nível de pressão atual
  altura = calcular_altura(PA, k, level, g)
  # Adiciona a coluna de altura
  horas_df['Altitude_m'] = altura

  # Adiciona o DataFrame do nível de pressão atual ao DataFrame combinado
  combined_df = pd.concat([combined_df, horas_df], join='outer', ignore_index=True)

  # Fecha o arquivo NetCDF
  ds.close()


In [ ]:
# Ajusta a ordem das colunas
combined_df = combined_df[['Plataforma', 'Nível_de_Pressão_hPa', 'Altitude_m', 'Estação_do_Ano', 'Horário_Brasília', 'Horário_UTC',
                           'Velocidade_Vento_u_m/s', 'Velocidade_Vento_v_m/s', 'Velocidade_Vento_media_m/s', 'Temperatura_K', 'Temperatura_C']]

# Salvar o DataFrame combinado como um arquivo CSV
csv_filename = f'2023_DataFrame.csv'
combined_df.to_csv(csv_filename, encoding='utf-8', index=True)

# Salvar o DataFrame combinado como um arquivo Excel
excel_filename = f'2023_DataFrame.xlsx'
combined_df.to_excel(excel_filename, index=True)